In [ ]:
import numpy as np
import cv2
from tqdm import tqdm
import time
import signal
from matplotlib import pyplot as plt

In [ ]:
%pylab inline
import pylab as pl
from IPython import display

In [ ]:
def preProcess_images(path,frame_nums):
    X = []
    for i in range(1500):
        im_full = cv2.imread(path + '/' 
                        + str(int(frame_nums[i])).zfill(4) + '.jpg')
        resized_image = cv2.resize(im_full, (64, 64))
        grayscale_image = np.mean(resized_image, axis = 2)
        normalized = grayscale_image/255
        oneD_vec = np.ravel(normalized)
        X.append(oneD_vec)
    return np.asarray(X)

def preProcess_angles(steering_angles):
    maxAngle = steering_angles.max()
    minAngle = steering_angles.min()
    new_angle = []
    for i in range(steering_angles.shape[0]):
        new_angle.append(((steering_angles[i] - minAngle)) / (maxAngle - minAngle))
    return np.asarray(new_angle)  

In [ ]:
student_file = 'sample_student'
path_to_training_images = 'data/training/images'
training_csv_file = 'data/training/steering_angles.csv'
path_to_testing_images = 'data/training/images'
testing_csv_file = 'data/training/steering_angles.csv'
time_limit = 600

In [ ]:
data = np.genfromtxt(training_csv_file, delimiter = ',')
frame_nums = data[:,0]
steering_angles = data[:,1]
print(steering_angles.shape)

In [ ]:
X = preProcess_images(path_to_training_images, frame_nums)
y = preProcess_angles(steering_angles)

In [ ]:
y = y.reshape((1500,1))

In [ ]:
X.shape,y.shape

In [ ]:
class Neural_Network(object):
    def __init__(self, Lambda=0):        
        #Define Hyperparameters
        self.inputLayerSize = 64*64
        self.outputLayerSize = 1
        self.hiddenLayerSize = 30
        
        #Weights (parameters)
        self.W1 = np.random.randn(self.inputLayerSize,self.hiddenLayerSize)
        self.W2 = np.random.randn(self.hiddenLayerSize,self.outputLayerSize)
        
        #Regularization Parameter:
        self.Lambda = Lambda
        
    def forward(self, X):
        #Propogate inputs though network
        self.z2 = np.dot(X, self.W1)
        self.a2 = self.sigmoid(self.z2)
        self.z3 = np.dot(self.a2, self.W2)
        yHat = self.sigmoid(self.z3) 
        return yHat
        
    def sigmoid(self, z):
        #Apply sigmoid activation function to scalar, vector, or matrix
        return 1/(1+np.exp(-z))
    
    def sigmoidPrime(self,z):
        #Gradient of sigmoid
        return np.exp(-z)/((1+np.exp(-z))**2)
    
    def costFunction(self, X, y):
        #Compute cost for given X,y, use weights already stored in class.
        self.yHat = self.forward(X)
        J = 0.5*sum((y-self.yHat)**2)/X.shape[0] + (self.Lambda/2)*(np.sum(self.W1**2)+np.sum(self.W2**2))
        return J
        
    def costFunctionPrime(self, X, y):
        #Compute derivative with respect to W and W2 for a given X and y:
        self.yHat = self.forward(X)
        
        delta3 = np.multiply(-(y-self.yHat), self.sigmoidPrime(self.z3))
        #Add gradient of regularization term:
        dJdW2 = np.dot(self.a2.T, delta3)/X.shape[0] + self.Lambda*self.W2
        
        delta2 = np.dot(delta3, self.W2.T)*self.sigmoidPrime(self.z2)
        #Add gradient of regularization term:
        dJdW1 = np.dot(X.T, delta2)/X.shape[0] + self.Lambda*self.W1
        
        return dJdW1, dJdW2
    
    #Helper functions for interacting with other methods/classes
    def getParams(self):
        #Get W1 and W2 Rolled into vector:
        params = np.concatenate((self.W1.ravel(), self.W2.ravel()))
        return params
    
    def setParams(self, params):
        #Set W1 and W2 using single parameter vector:
        W1_start = 0
        W1_end = self.hiddenLayerSize*self.inputLayerSize
        self.W1 = np.reshape(params[W1_start:W1_end], \
                             (self.inputLayerSize, self.hiddenLayerSize))
        W2_end = W1_end + self.hiddenLayerSize*self.outputLayerSize
        self.W2 = np.reshape(params[W1_end:W2_end], \
                             (self.hiddenLayerSize, self.outputLayerSize))
        
    def computeGradients(self, X, y):
        dJdW1, dJdW2 = self.costFunctionPrime(X, y)
        return np.concatenate((dJdW1.ravel(), dJdW2.ravel()))

In [ ]:
NN = Neural_Network(Lambda=0.0001)
grads = NN.computeGradients(X, y)
num_iterations = 5000
alpha = 1e-2
beta1= 0.5
beta2= 0.999
epsilon= 1e-05

m0 = np.zeros(len(grads)) #Initialize first moment vector
v0 = np.zeros(len(grads)) #Initialize second moment vector
t = 0.0

losses = [] #For visualization
RMSE_vec = []
mt = m0
vt = v0
fig1 = figure()
for i in range(num_iterations):
    if i%50==0:
        alpha = alpha * 0.1
    t += 1
    grads = NN.computeGradients(X = X, y = y)
    mt = beta1*mt + (1-beta1)*grads
    vt = beta2*vt + (1-beta2)*grads**2
    mt_hat = mt/(1-beta1**t)
    vt_hat = vt/(1-beta2**t)

    params = NN.getParams()
    new_params = params - alpha*mt_hat/(np.sqrt(vt_hat)+epsilon)
    NN.setParams(new_params)

    losses.append(NN.costFunction(X = X, y = y))   
    
    angles = NN.forward(X)
    new_a = []
    for a in range(1500):
        new_a.append((angles[a]) * (steering_angles.max() - steering_angles.min()) + steering_angles.min())
    
    RMSE = np.sqrt(np.mean((np.array(new_a) - steering_angles)**2))
    print("Epoch: {} -> Loss: {}".format(i, RMSE))
#     RMSE_vec.append(RMSE)
#     ## Clear the figure
#     if i%10==0:
#         pl.plot(RMSE_vec)
#         display.clear_output(wait=True)
#         display.display(pl.gcf())

In [ ]:
angles.shape

In [ ]:
T= NN.forward(im_full)
T

In [ ]:
nn,loss = trainFunc(train_im,train_an)

In [ ]:
nn.forward(im_full)

In [ ]:
figure(0,(20,20))
plot(loss)